# Conditions and to do's:
---
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
    - San Francisco: Financial District, South Of Market or Mission Bay
- Everyone in the company is between 25 and 40, give them some place to go party.
    - San Francisco: South Of Market -> 13 possible offices
- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
    - `[-122.394523, 37.764726]` in Potrero Hill  
---
To do:
- 30% of the company staff have at least 1 child.
    - To do: look for schools in/near South Of Market
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
    - To do: look for Starbucks in/near South Of Market
- Account managers need to travel a lot.
    - To do: look for renting car places in/near South Of Market
- The CEO is vegan.
    - To do: look for vegan places in/near South of Market
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
    - To do: look for a basketball stadium and check the distance to the possible offices
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.
    - To do: look for a pet hairdresser

# Libraries

In [1]:
import os
import requests
import json
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from dotenv import load_dotenv
import time
import math

# Requests

In [ ]:
def getCoordinates(address):
    url_geocode = f"https://geocode.xyz/{address}?json=1"
    res = requests.get(url_geocode)
    try:
        return res.json()
    except:
        time.sleep(4)
        getCoordinates(address)

In [ ]:
res = getCoordinates("San Francisco")
res

In [ ]:
lat, lon = res["alt"]["loc"]["latt"], res["alt"]["loc"]["longt"]
map_SF = Map(location = [lat, lon], zoom_start = 12)
map_SF

In [ ]:
# These coordinates are not very accurate for the neighborhood of interest. I'm using the one found in wikipedia:
lat = 37.7772222
lon = -122.4111111

In [19]:
load_dotenv()

True

In [20]:
token = os.getenv("token")

In [21]:
#Size of South of Market: it is aproximately 2.3 miles long -> 3.7 km long

In [22]:
def request_4sq(venue, lat, lon, radius = 3700, sort_by = "DISTANCE", limit = 20):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&radius={radius}&sort={sort_by}&limit={limit}"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    try:
        return requests.get(url, headers = headers).json()
    except:
        print("Request not found")

In [23]:
school_request = request_4sq("school", lat, lon)

# Distances

In [56]:
def distances(coordinates_1, coordinates_2):
    R = 6371000 # radius of Earth in meters
    lon_1, lat_1 = coordinates_1
    lon_2, lat_2 = coordinates_2
    phi_1 = math.radians(lat_1)
    phi_2 = math.radians(lat_2)
    delta_phi = math.radians(lat_2 - lat_1)
    delta_lambda = math.radians(lon_2 - lon_1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    km = round(R * c / 1000.0, 3)
    return km

# Weighing priorities

According to the number of people affected by their expressed needs, they are prioritised:
- Everyone in the company is between 25 and 40, give them some place to go party. **DONE**
- 30% of the company staff (29 people) have at least 1 child. **school_df, PRIORITY 1**
- 20 Designers : Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design. **designSF_coordinates, PRIORITY 2**
- 20 Account Managers: Account managers need to travel a lot. **cars_df, PRIORITY 3**
- 15 Data Engineers
- 10 Frontend Developers, 5 Backend Developers: Developers like to be near successful tech startups that have raised at least 1 Million dollars. **DONE**
- 10 Executives: Executives like Starbucks A LOT. Ensure there's a starbucks not too far. **sctarbucks_df, PRIORITY 4**
- 5 UI/UX Engineers
- 1 Maintenance guy that loves basketball. **courts_df, PRIORITY 5**
- 1 CEO/President: The CEO is vegan. **vegan_df, PRIORITY 6**
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away. **dogs_df, PRIORITY 7**

## Priority 1: Schools

In [29]:
school_request["results"][3]["categories"][0]["name"]

'Daycare'

In [30]:
school_categ = [i["categories"][0]["name"] for i in school_request["results"]]
print(school_categ)

['College and University', 'Preschool', 'Technology Business', 'Daycare', 'Education', 'Language School', 'Elementary School', 'Preschool', 'College Academic Building', 'Music School', 'College and University', 'Daycare', 'Law School', 'Education', 'College Academic Building', 'Preschool', 'Non-Profit Organization', 'Education', 'Education', 'Public Art']


In [31]:
school_lat = [i["geocodes"]["main"]["latitude"] for i in school_request["results"]]
school_lon = [i["geocodes"]["main"]["longitude"] for i in school_request["results"]]

In [112]:
school_name = [i["name"] for i in school_request["results"]]

In [33]:
school_open = [i["closed_bucket"] for i in school_request["results"]]
print(school_open)

['LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyClosed', 'VeryLikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'VeryLikelyOpen', 'Unsure', 'LikelyOpen', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'VeryLikelyOpen', 'Unsure', 'LikelyClosed', 'Unsure', 'LikelyOpen', 'LikelyOpen', 'Unsure', 'LikelyOpen']


In [116]:
df_schools = pd.DataFrame({"school_name":school_name, "school_categ": school_categ, "school_lat": school_lat, "school_lon": school_lon, "school_open": school_open})

In [117]:
df_schools

,school_name,school_categ,school_lat,school_lon,school_open
0,Bradfield School of Computer Science,College and University,37.777085,-122.410024,LikelyOpen
1,KinderCare Learning Center,Preschool,37.779426,-122.411882,VeryLikelyOpen
2,Drawn to Scale HQ / Beauty School,Technology Business,37.779378,-122.410374,VeryLikelyClosed
3,C5 Children's School,Daycare,37.780990,-122.419214,VeryLikelyOpen
4,Bay Area Motorcycle School,Education,37.774363,-122.413160,LikelyOpen
5,Ukrainian School of San Francisco / Українська...,Language School,37.776338,-122.406969,VeryLikelyOpen
6,Bessie Carmichael Elementary School,Elementary School,37.776344,-122.406468,VeryLikelyOpen
7,Presidio Knolls School,Preschool,37.773203,-122.413809,Unsure
8,California Institute of Integral Studies,College Academic Building,37.774589,-122.416184,LikelyOpen
9,John Scott School Of Voice,Music School,37.781455,-122.407558,Unsure


In [127]:
indexes = df_schools["school_categ"][df_schools["school_categ"].isin(["College and University", "Technology Business", "Language School", 
                                               "College Academic Building", "Music School", "College and University", "Law School",
                                               "College Academic Building", "Public Art"])].index
df_schools.drop(indexes, inplace=True)
indexes = df_schools["school_open"][df_schools["school_open"].isin(["Unsure", "LikelyClosed"])].index
df_schools.drop(indexes, inplace=True)
df_schools
indexes = df_schools["school_name"][df_schools["school_name"].isin(["Bay Area Motorcycle School", "Tenderloin After-School Program Center", "SFUSD-Van Ness"])].index
df_schools.drop(indexes, inplace=True)
df_schools.drop(columns = "school_open", axis = 1, inplace=True)
df_schools

,school_name,school_categ,school_lat,school_lon
1,KinderCare Learning Center,Preschool,37.779426,-122.411882
3,C5 Children's School,Daycare,37.780990,-122.419214
6,Bessie Carmichael Elementary School,Elementary School,37.776344,-122.406468
11,Love & Learn Nursery School,Daycare,37.773386,-122.414823


### School distance

In [128]:
list_offices = [{'type': 'Point', 'coordinates': [-122.391276, 37.7815]},
                {'type': 'Point', 'coordinates': [-122.3921923, 37.781381]},
                {'type': 'Point', 'coordinates': [-122.4097127, 37.78205]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.405783, 37.785435]},
                {'type': 'Point', 'coordinates': [-122.408109, 37.783521]},
                {'type': 'Point', 'coordinates': [-122.3920816, 37.7828482]},
                {'type': 'Point', 'coordinates': [-122.3920816, 37.7828482]},
                {'type': 'Point', 'coordinates': [-122.39586, 37.785766]}]

In [129]:
school_coordinates = [[row["school_lat"], row["school_lon"]] for index, row in df_schools.iterrows()]
school_coordinates

[[37.779426, -122.411882],
 [37.78099, -122.419214],
 [37.776344, -122.406468],
 [37.773386, -122.414823]]

In [130]:
school_distances = [{"office":j, "school":i, "distance": distances(i, [j["coordinates"][1],j["coordinates"][0]])} for j in list_offices for i in school_coordinates]
df_school_distance = pd.DataFrame(school_distances).sort_values(by = "distance", ascending = True)
df_school_distance

,office,school,distance
8,"{'type': 'Point', 'coordinates': [-122.4097127...","[37.779426, -122.411882]",0.287
38,"{'type': 'Point', 'coordinates': [-122.408109,...","[37.776344, -122.406468]",0.465
36,"{'type': 'Point', 'coordinates': [-122.408109,...","[37.779426, -122.411882]",0.485
10,"{'type': 'Point', 'coordinates': [-122.4097127...","[37.776344, -122.406468]",0.496
14,"{'type': 'Point', 'coordinates': [-122.405783,...","[37.776344, -122.406468]",0.547
26,"{'type': 'Point', 'coordinates': [-122.405783,...","[37.776344, -122.406468]",0.547
34,"{'type': 'Point', 'coordinates': [-122.405783,...","[37.776344, -122.406468]",0.547
22,"{'type': 'Point', 'coordinates': [-122.405783,...","[37.776344, -122.406468]",0.547
18,"{'type': 'Point', 'coordinates': [-122.405783,...","[37.776344, -122.406468]",0.547
30,"{'type': 'Point', 'coordinates': [-122.405783,...","[37.776344, -122.406468]",0.547


In [131]:
df_school_distance[df_school_distance["distance"] <= 0.5]["office"].value_counts()

office
{'type': 'Point', 'coordinates': [-122.4097127, 37.78205]}    2
{'type': 'Point', 'coordinates': [-122.408109, 37.783521]}    2
Name: count, dtype: int64

In [132]:
x = pd.DataFrame(df_school_distance[df_school_distance["distance"] <= 0.5]["office"].value_counts())
x.reset_index(inplace=True)

In [133]:
list_offices_updated = [row["office"] for index, row in x.iterrows()]
list_offices_updated

[{'type': 'Point', 'coordinates': [-122.4097127, 37.78205]},
 {'type': 'Point', 'coordinates': [-122.408109, 37.783521]}]

## Priority 2: design company

In [144]:
designSF_coordinates = [{'offices': [{'latitude': 37.764726, 'longitude': -122.394523}]}]
designSF_latlon = [designSF_coordinates[0]["offices"][0]["latitude"], designSF_coordinates[0]["offices"][0]["longitude"]]

In [151]:
design_distance = [{"office":i, "distance": distances(designSF_latlon, [i["coordinates"][1],i["coordinates"][0]])} for i in list_offices_updated]
design_distance
df_design_distance = pd.DataFrame(design_distance).sort_values(by = "distance", ascending = True)

In [152]:
df_design_distance = df_design_distance.head(1)
df_design_distance

,office,distance
1,"{'type': 'Point', 'coordinates': [-122.408109,...",1.881


# Final office:

In [137]:
for index, row in df_design_distance.iterrows():
    print(row["office"])

{'type': 'Point', 'coordinates': [-122.408109, 37.783521]}


In [138]:
coordinates = [row["office"] for index, row in df_design_distance.iterrows()][0]["coordinates"]
coordinates

[-122.408109, 37.783521]

In [139]:
rents = pd.read_csv("../data/officesSF.csv", encoding = "unicode_escape")
rents[rents["positions"]== str(coordinates)]

,employees,lat,lon,address,positions
143,112,37.783521,-122.408109,901 Market Street,"[-122.408109, 37.783521]"


In [140]:
# The school closer to the office would be:
final_school = df_school_distance[df_school_distance["office"] == [row["office"] for index, rows in df_design_distance.iterrows()][0]].sort_values(by = "distance", ascending = True).head(1)
final_school

,office,school,distance
38,"{'type': 'Point', 'coordinates': [-122.408109,...","[37.776344, -122.406468]",0.465


In [141]:
coordinates_school = [row["school"] for index, row in final_school.iterrows()][0]
coordinates_school

[37.776344, -122.406468]

In [142]:
df_schools[df_schools["school_lat"] == coordinates_school[0]]

,school_name,school_categ,school_lat,school_lon
6,Bessie Carmichael Elementary School,Elementary School,37.776344,-122.406468
